In [398]:
import pandas as pd
import numpy as np
from itertools import product,chain
from paramsearch import paramsearch
import pandas_profiling as pp

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import matthews_corrcoef
from scipy.stats import rankdata

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import catboost as cb
from sklearn.pipeline import Pipeline

In [12]:
train = pd.read_csv('train.csv', sep = '\t', index_col = 'Unnamed: 0')
test = pd.read_csv('test.csv', sep = '\t', index_col = 'Unnamed: 0')

train.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True,axis = 1)
test.drop(['140', '152', '160', '164','305', '303', '301', '193', '192', '186', '150', '141', '138', '130'],inplace = True, axis = 1)


X_train_old, y_train_old = train[train.columns[1:]], train['0']
X_test_old, y_test_old = test[test.columns[1:]], test['0'].fillna(0, inplace = True)




# Bagging on log-reg
best parameters were previously computed

In [56]:
lg = LogisticRegression(class_weight = 'balanced', C = 0.052499)
bg = BaggingClassifier(lg, random_state=42, n_estimators = 5 ,max_samples = 0.9, max_features = 1.0)
bg.fit(X_train_old, y_train_old)



BaggingClassifier(base_estimator=LogisticRegression(C=0.052499, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.9, n_estimators=5, n_jobs=1, oob_score=False,
         random_state=42, verbose=0, warm_start=False)

In [57]:
y_bg = bg.predict(X_test_old)

# random forest

In [59]:
rfc = RandomForestClassifier(random_state=42, n_jobs=-1 , class_weight = 'balanced', max_features = 100, n_estimators = 500, max_depth=90, min_samples_leaf= 1)
rfc.fit(X_train_old, y_train_old)

rfc.fit(X_train_old, y_train_old)
y_rfc = rfc.predict(X_test_old)


# ExtraTrees

In [ ]:
skf = StratifiedKFold(n_splits=4, random_state=42)

parameters = {'max_features': [17, 30, 35], 'min_samples_leaf': [5, 7, 10], 'max_depth': [15, 20]}
etc = ExtraTreesClassifier(n_estimators=150, random_state=42, n_jobs=-1, class_weight = 'balanced')
etcv = GridSearchCV(etc, parameters, n_jobs=-1, cv=skf, verbose=2, scoring='roc_auc')
etcv.fit(X_train_old, y_train_old)


In [30]:
etcv.best_estimator_, etcv.best_score_

(ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
            criterion='gini', max_depth=20, max_features=30,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=7,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=150, n_jobs=-1, oob_score=False, random_state=42,
            verbose=0, warm_start=False), 0.737687976105537)

In [31]:
best_xrf = etcv.best_estimator_
best_xrf.fit(X_train_old, y_train_old)
y_test = best_xrf.predict(X_test_old)

#header = ['_VAL_']
#test1 = test['0'].to_csv('bag_2.csv',sep =',', header = header, index_label = ['_ID_']).values 

print(np.sum(y_test)/len(y_test))
#print(np.sum(np.abs(y_test - test1))/len(y_test))

0.2525204032645223


In [32]:
xrf = etcv.best_estimator_
y_xrf = best_xrf.predict(X_test_old)

# XGBoost

In [ ]:
spw = (len(y_train_old) - np.sum(y_train_old))/np.sum(y_train_old)
gbm = xgb.XGBClassifier(n_estimators = 150, subsample=0.8, colsample_bytree=0.8, objective = 'binary:logistic', scale_pos_weight = spw)

dmatrix = xgb.DMatrix(data=X_train_old, label=y_train_old)

#spw = (len(y_train_old) - np.sum(y_train_old))/np.sum(y_train_old)

parameters = {'learning_rate' : [0.1, 0.05], 'max_depth' : [5, 10], 'reg_lambda' : [1]}
gbmcv = GridSearchCV(estimator = gbm, param_grid=parameters, scoring = 'roc_auc', cv = skf, verbose = 2, n_jobs = -1)
gbmcv.fit(X_train_old, y_train_old)

In [48]:
gbmcv.best_estimator_, gbmcv.best_score_

(XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
        gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=5,
        min_child_weight=1, missing=None, n_estimators=150, nthread=-1,
        objective='binary:logistic', reg_alpha=0, reg_lambda=1,
        scale_pos_weight=4.624193250968099, seed=0, silent=True,
        subsample=0.8), 0.7384816015468296)

In [49]:
best_gbm = gbmcv.best_estimator_
best_gbm.fit(X_train_old, y_train_old)
y_test = best_gbm.predict(X_test_old)

#header = ['_VAL_']
#test1 = test['0'].to_csv('bag_2.csv',sep =',', header = header, index_label = ['_ID_']).values 

print(np.sum(y_test)/len(y_test))
#print(np.sum(np.abs(y_test - test1))/len(y_test))

0.3170907345175228


/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [61]:
y_gbm = best_gbm.predict(X_test_old)


/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Voting 1
majority

In [197]:
arr = y_bg + y_rfc + y_xrf + y_gbm

In [198]:
arr = np.where(arr < 2 , 0, arr)
arr = np.where(arr >= 2 , 1, arr)

np.sum(arr)/len(arr)

0.3050888142102736

In [199]:
test['0'] = arr
header = ['_VAL_']
test['0'].to_csv('ensemble_4.csv',sep =',', header = header, index_label = ['_ID_']) 

# Voting 2 ( Bagging, Extra trees, Random forest, Xgboost)
rank

CV -score 0.7457
LB -score 0.691

In [110]:
y_gbm_pr = best_gbm.predict_proba(X_test_old)
y_xrf_pr = best_xrf.predict_proba(X_test_old)
y_rfc_pr = rfc.predict_proba(X_test_old)
y_bg_pr = bg.predict_proba(X_test_old)

In [130]:
rfc_predictions = []
gbm_predictions = []
xrf_predictions = []
bg_predictions = []


for train, val in skf.split(X_train_old, y_train_old):
    best_gbm.fit(X_train_old.iloc[train], y_train_old[train])
    best_xrf.fit(X_train_old.iloc[train], y_train_old[train])
    rfc.fit(X_train_old.iloc[train], y_train_old[train])
    bg.fit(X_train_old.iloc[train], y_train_old[train])
    
    
    bg_predictions.append([y_train_old[val], bg.predict_proba(X_train_old.iloc[val])[:,1]])
    rfc_predictions.append([y_train_old[val], rfc.predict_proba(X_train_old.iloc[val])[:,1]])
    gbm_predictions.append([y_train_old[val], best_gbm.predict_proba(X_train_old.iloc[val])[:,1]])
    xrf_predictions.append([y_train_old[val], best_xrf.predict_proba(X_train_old.iloc[val])[:,1]])

In [165]:
np.mean([roc_auc_score(rf_p[0], rankdata(rf_p[1]) + rankdata(et_p[1]) + rankdata(gb_p[1]) + rankdata(bg_p[1])) for rf_p, et_p, gb_p, bg_p in zip(rfc_predictions, xrf_predictions, gbm_predictions, bg_predictions)])

0.7457898794387926

In [160]:
def Find_Optimal_Cutoff(target, predicted):
    """ Find the optimal probability cutoff point for a classification model related to event rate
    Parameters
    ----------
    target : Matrix with dependent or target data, where rows are observations

    predicted : Matrix with predicted data, where rows are observations

    Returns
    -------     
    list type, with optimal cutoff value

    """
    fpr, tpr, threshold = roc_curve(target, predicted)
    i = np.arange(len(tpr)) 
    roc = pd.DataFrame({'tf' : pd.Series(tpr-(1-fpr), index=i), 'threshold' : pd.Series(threshold, index=i)})
    roc_t = roc.ix[(roc.tf-0).abs().argsort()[:1]]

    return list(roc_t['threshold']) 



In [166]:
best_gbm.fit(X_train_old, y_train_old)
best_xrf.fit(X_train_old, y_train_old)
rfc.fit(X_train_old, y_train_old)
bg.fit(X_train_old, y_train_old)

rfc_predictions_1 = []
gbm_predictions_1 = []
xrf_predictions_1 = []
bg_predictions_1 = []

bg_predictions_1.append(bg.predict_proba(X_train_old)[:,1])
rfc_predictions_1.append(rfc.predict_proba(X_train_old)[:,1])
gbm_predictions_1.append(best_gbm.predict_proba(X_train_old)[:,1])
xrf_predictions_1.append(best_xrf.predict_proba(X_train_old)[:,1])

In [181]:
t = (rankdata(rfc_predictions_1[0]) + rankdata(xrf_predictions_1[0]) + rankdata(gbm_predictions_1[0]) + rankdata(bg_predictions_1[0]))

tr = Find_Optimal_Cutoff(y_train_old, t)[0]
print(tr)

82073.0


/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [184]:
coef = tr/(max(t) - min(t))


bg_predictions_1.append(bg.predict_proba(X_test_old)[:,1])
rfc_predictions_1.append(rfc.predict_proba(X_test_old)[:,1])
gbm_predictions_1.append(best_gbm.predict_proba(X_test_old)[:,1])
xrf_predictions_1.append(best_xrf.predict_proba(X_test_old)[:,1])

t_1 = (rankdata(rfc_predictions_1[0]) + rankdata(xrf_predictions_1[0]) + rankdata(gbm_predictions_1[0]) + rankdata(bg_predictions_1[0]))

new_thr = coef * (max(t_1) - min(t_1)) + min(t_1)

In [194]:
t_1 = np.where(t_1 < new_thr , 0, t_1)
t_1 = np.where(t_1 >= new_thr , 1, t_1)
print(np.sum(t_1)/len(t_1))

0.30196831493038884


In [211]:
test['0'] = t_1
header = ['_VAL_']
test['0'].to_csv('ensemble_vote2.csv',sep =',', header = header, index_label = ['_ID_']) 

дальше я тренирую новые модели(два Dart XGB, и два катбуста)

# Dart XGB

In [236]:
num_round_1 = 150

params_1 = {'booster': 'dart',
         'max_depth': 10, 'learning_rate': 0.05,
         'objective': 'binary:logistic', 'silent': True,
         'sample_type': 'weighted',
         'normalize_type': 'tree',
         'rate_drop': 0.25,
         'subsample':0.7, 'colsample_bytree':0.7, 'eval_metric':'auc'}

xgb_1 = xgb.train(params_1, dtrain, num_boost_round = num_round_1, verbose_eval = True)

preds_1 = xgb_1.predict(dtest)

In [245]:
num_round_2 = 150

params_2 = {'booster': 'dart',
         'max_depth': 7, 'learning_rate': 0.08,
         'objective': 'binary:logistic', 'silent': True,
         'sample_type': 'weighted',
         'normalize_type': 'tree',
         'rate_drop': 0.25,
         'subsample':0.6, 'colsample_bytree':0.6, 'eval_metric':'auc'}

xgb_2 = xgb.train(params_2, dtrain, num_boost_round = num_round_2, verbose_eval = True)

preds_2 = xgb_2.predict(dtest)

In [246]:
print(np.corrcoef(preds_1, preds_2)[0][1])
print('preds_1')
print(np.corrcoef(preds_1, y_gbm)[0][1])
print(np.corrcoef(preds_1, y_rfc)[0][1])
print(np.corrcoef(preds_1, y_xrf)[0][1])
print(np.corrcoef(preds_1, y_bg)[0][1])
print('preds_2')
print(np.corrcoef(preds_2, y_gbm)[0][1])
print(np.corrcoef(preds_2, y_rfc)[0][1])
print(np.corrcoef(preds_2, y_xrf)[0][1])
print(np.corrcoef(preds_2, y_bg)[0][1])




0.9734425194676225
preds_1
0.7316078456403736
0.5924411740445731
0.7652934700062765
0.6695967345036121
preds_2
0.7484585306956676
0.564183240383969
0.7715719238095538
0.6859181318761481


# Catboost

In [421]:

category_cols = [i for i in X_train_old.columns if len(X_train_old[i].value_counts()) == 2 ]

cat_dims = [X_train_old.columns.get_loc(i) for i in category_cols[:-1]] 


# this function does 3-fold crossvalidation with catboostclassifier          
def crossvaltest(params, train_set, train_label, cat_dims, n_splits=4, cross_val = skf):
    #kf = KFold(n_splits=n_splits,shuffle=True)
    kf = skf
    res = []
    for train_index, test_index in kf.split(train_set, train_label):
        train = train_set.iloc[train_index,:]
        test = train_set.iloc[test_index,:]

        labels = train_label.ix[train_index]
        test_labels = train_label.ix[test_index]

        clf = cb.CatBoostClassifier(**params)
        clf.fit(train, np.ravel(labels), cat_features=cat_dims, verbose = True)

        res.append(np.mean(clf.predict(test)==np.ravel(test_labels)))
    
    print('res = ', np.mean(res))
    return np.mean(res)


# this function runs grid search on several parameters
def catboost_param_tune(params, train_set, train_label, cat_dims=None, n_splits=4, cross_val = skf):
    ps = paramsearch(params)
    # search 'border_count', 'l2_leaf_reg' etc. individually 
    #   but 'iterations','learning_rate' together
    for prms in chain(#ps.grid_search(['border_count']),
                      #ps.grid_search(['ctr_border_count']),
                      ps.grid_search(['l2_leaf_reg']),
                      ps.grid_search(['iterations','learning_rate']),
                      ps.grid_search(['depth'])):
        res = crossvaltest(prms, train_set, train_label, cat_dims, n_splits, skf)
        # save the crossvalidation result so that future iterations can reuse the best parameters
        ps.register_result(res,prms)
        #print(res,prms,s,'best:',ps.bestscore(),ps.bestparam())
        print('best:',ps.bestscore(),ps.bestparam())
    return ps.bestparam()




In [ ]:
params = {'depth':[5, 7],
          'iterations':[150],
          'learning_rate':[0.05], 
          'l2_leaf_reg':[1, 3],
          #'border_count':[32],
          #'ctr_border_count':[20],
          'thread_count':4,
          'scale_pos_weight':spw,
          'eval_metric':'AUC'}

bestparams = catboost_param_tune(params, X_train_old, y_train_old, cat_dims, 4, skf)



OUT: best: 0.8267212928269888 {'thread_count': 4, 'iterations': 150, 'depth': 5, 'l2_leaf_reg': 3, 'scale_pos_weight': 4.624193250968099, 'learning_rate': 0.05, 'eval_metric': 'AUC'}

In [309]:
clf_cb1 = cb.CatBoostClassifier(**bestparams)
clf_cb1.fit(X_train_old, np.ravel(y_train_old), cat_features = cat_dims)
res_cb1 = clf_cb1.predict_proba(X_test_old)[:,1]

In [326]:
print(np.corrcoef(res_cb1, y_gbm)[0][1])


0.7545372242555904


# Catboost2

In [327]:
params2 = {'depth':[3],
          'iterations':[200, 250],
          'learning_rate':[0.1, 0.05], 
          'l2_leaf_reg':[5, 7],
          #'border_count':[32],
          #'ctr_border_count':[20],
          'thread_count':4,
          'scale_pos_weight':spw,
          'eval_metric':'AUC'}

In [ ]:
bestparams2 = catboost_param_tune(params2, X_train_old, y_train_old, cat_dims, 4, skf)



best: 0.8276066116979741 {'thread_count': 4, 'iterations': 250, 'depth': 3, 'learning_rate': 0.1, 'scale_pos_weight': 4.624193250968099, 'l2_leaf_reg': 7, 'eval_metric': 'AUC'}

In [330]:
clf_cb2 = cb.CatBoostClassifier(**bestparams2)
clf_cb2.fit(X_train_old, np.ravel(y_train_old), cat_features = cat_dims)
res_cb2 = clf_cb2.predict_proba(X_test_old)[:,1]

In [332]:
print(np.corrcoef(res_cb2, preds_2)[0][1])

print(np.corrcoef(res_cb1, res_cb2)[0][1])



0.9357028235171487
0.9796548285297352


# Ensembling 1
на всех фичах и еще предикты моделей

catboost


In [ ]:
rfc_predictions = []
gbm_predictions = []
xrf_predictions = []
bg_predictions = []
xgb_1_predictions = []
xgb_2_predictions = []
clf_cb1_predictions = []
clf_cb2_predictions = []




for train, val in skf.split(X_train_old, y_train_old):
    
    best_gbm.fit(X_train_old.iloc[train], y_train_old[train])
    best_xrf.fit(X_train_old.iloc[train], y_train_old[train])
    rfc.fit(X_train_old.iloc[train], y_train_old[train])
    bg.fit(X_train_old.iloc[train], y_train_old[train])
    

    dtrain = xgb.DMatrix(data = X_train_old.iloc[train], label = y_train_old.iloc[train])
    dtest = xgb.DMatrix(X_train_old.iloc[val])

    xgb_1 = xgb.train(params_1, dtrain, num_boost_round = num_round_1, verbose_eval = False)
    xgb_2 = xgb.train(params_2, dtrain, num_boost_round = num_round_2, verbose_eval = False)  
    clf_cb1.fit(X_train_old.iloc[train], np.ravel(y_train_old[train]),  cat_features = cat_dims)
    clf_cb2.fit(X_train_old.iloc[train], np.ravel(y_train_old[train]), cat_features = cat_dims)
    
    preds1 = xgb_1.predict(dtest)
    preds2 = xgb_2.predict(dtest)    
    
    bg_predictions.append([y_train_old[val], bg.predict_proba(X_train_old.iloc[val])[:,1]])
    rfc_predictions.append([y_train_old[val], rfc.predict_proba(X_train_old.iloc[val])[:,1]])
    gbm_predictions.append([y_train_old[val], best_gbm.predict_proba(X_train_old.iloc[val])[:,1]])
    xrf_predictions.append([y_train_old[val], best_xrf.predict_proba(X_train_old.iloc[val])[:,1]])
    
    xgb_1_predictions.append([y_train_old[val], preds1])
    xgb_2_predictions.append([y_train_old[val], preds2])
    clf_cb1_predictions.append([y_train_old[val], clf_cb1.predict_proba(X_train_old.iloc[val])[:,1]])
    clf_cb2_predictions.append([y_train_old[val], clf_cb2.predict_proba(X_train_old.iloc[val])[:,1]])


    
    
    

In [363]:
best_gbm.fit(X_train_old, y_train_old)
best_xrf.fit(X_train_old, y_train_old)
rfc.fit(X_train_old, y_train_old)
bg.fit(X_train_old, y_train_old)
    
dtrain = xgb.DMatrix(data = X_train_old, label = y_train_old)
dtest = xgb.DMatrix(X_test_old)

xgb_1 = xgb.train(params_1, dtrain, num_boost_round = num_round_1, verbose_eval = False)
xgb_2 = xgb.train(params_2, dtrain, num_boost_round = num_round_2, verbose_eval = False)  
clf_cb1.fit(X_train_old, np.ravel(y_train_old),  cat_features = cat_dims)
clf_cb2.fit(X_train_old, np.ravel(y_train_old), cat_features = cat_dims)
rfc_preds_test = rfc.predict_proba(X_test_old)[:,1]
xrf_preds_test = best_xrf.predict_proba(X_test_old)[:,1]
gbm_preds_test = best_gbm.predict_proba(X_test_old)[:,1]
bg_preds_test = bg.predict_proba(X_test_old)[:,1]

xgb_1_preds_test = xgb_1.predict(dtest)
xgb_2_preds_test = xgb_2.predict(dtest)
clf_cb_1_preds_test = clf_cb1.predict_proba(X_test_old)[:,1]
clf_cb_2_preds_test = clf_cb2.predict_proba(X_test_old)[:,1]


X_test_old["rfc_preds"] = rfc_preds_test
X_test_old["xrf_preds"] = xrf_preds_test
X_test_old["gbm_preds"] = gbm_preds_test
X_test_old["bg_preds"] = bg_preds_test

X_test_old["xgb_1_preds"] = xgb_1_preds_test
X_test_old["xgb_2_preds"] = xgb_2_preds_test
X_test_old["clf_cb_1_preds"] = clf_cb_1_preds_test
X_test_old["clf_cb_2_preds"] = clf_cb_2_preds_test





X_train_old["rfc_preds"] = 0
X_train_old["xrf_preds"] = 0
X_train_old["gbm_preds"] = 0
X_train_old["bg_preds"] = 0

X_train_old["xgb_1_preds"] = 0
X_train_old["xgb_2_preds"] = 0
X_train_old["clf_cb_1_preds"] = 0
X_train_old["clf_cb_2_preds"] = 0



for i, fold in enumerate(skf.split(X_train_old, y_train_old)):
    train, val = fold[0], fold[1]
    
    
    
    X_train_old.iloc[val, -8] = rfc_predictions[i][1]
    X_train_old.iloc[val, -7] = xrf_predictions[i][1]
    X_train_old.iloc[val, -6] = gbm_predictions[i][1]
    X_train_old.iloc[val, -5] = bg_predictions[i][1]
    
    X_train_old.iloc[val, -4] = xgb_1_predictions[i][1]
    X_train_old.iloc[val, -3] = xgb_2_predictions[i][1]
    X_train_old.iloc[val, -2] = clf_cb1_predictions[i][1]
    X_train_old.iloc[val, -1] = clf_cb2_predictions[i][1]



In [ ]:
params3 = {'depth':[2, 3],
          'iterations':[200, 300],
          'learning_rate':[0.1], 
          'l2_leaf_reg':[7],
          #'border_count':[32],
          #'ctr_border_count':[20],
          'thread_count':4,
          'scale_pos_weight':spw,
          'eval_metric':'AUC'}

bestparams3 = catboost_param_tune(params3, X_train_old, y_train_old, cat_dims, 4, skf)




OUT: best: 0.8298689284459584 {'thread_count': 4, 'iterations': 300, 'depth': 2, 'l2_leaf_reg': 7, 'scale_pos_weight': 4.624193250968099, 'learning_rate': 0.1, 'eval_metric': 'AUC'}

In [366]:
clf_cb3 = cb.CatBoostClassifier(**bestparams3)
clf_cb3.fit(X_train_old, np.ravel(y_train_old), cat_features = cat_dims)

res_cb3 = clf_cb3.predict_proba(X_test_old)[:,1]

yt = clf_cb3.predict_proba(X_train_old)[:,1]
tr = Find_Optimal_Cutoff(y_train_old, yt)[0]


res_cb3 = np.where(res_cb3 < tr , 0, res_cb3)
res_cb3 = np.where(res_cb3 >= tr , 1, res_cb3)

# Ensembling 2 
на всех фичах и еще предикты моделей

extra trees

он и оказался лучшим на лидерборде среди всех трех ансамблей, CV - 0.7467, LB - 0.6868

In [ ]:
parameters = {'max_features': [5, 15, 25], 'min_samples_leaf': [1, 3, 5], 'max_depth': [5,15,20], 'class_weight':['balanced']}

etc = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)
etcv = GridSearchCV(etc, parameters, n_jobs=-1, cv=skf, verbose=2, scoring='roc_auc')
etcv.fit(X_train_old, y_train_old)

In [383]:
etcv.best_score_

0.7467941483460391

In [386]:
best_et = etcv.best_estimator_
best_et.fit(X_train_old, y_train_old)
res_et = best_et.predict(X_test_old)

yt = best_et.predict_proba(X_train_old)[:,1]
tr = Find_Optimal_Cutoff(y_train_old, yt)[0]

yt = np.where(yt < tr , 0, yt)
yt = np.where(yt >= tr , 1, yt)
print(np.sum(yt)/len(yt))


0.30327868852459017


/Users/mefkov/miniconda3/envs/py3/lib/python3.5/site-packages/ipykernel/__main__.py:17: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [387]:
res_et = np.where(res_et < tr , 0, res_et)
res_et = np.where(res_et >= tr , 1, res_et)
print(np.sum(res_et)/len(res_et))

0.2695631301008161


# Ensembling 2 

только на предиктах других моделей

(cat boost)

In [ ]:
params4 = {'depth':[2, 3],
          'iterations':[150, 250],
          'learning_rate':[0.1], 
          'l2_leaf_reg':[7],
          #'border_count':[32],
          #'ctr_border_count':[20],
          'thread_count':4,
          'scale_pos_weight':spw,
          'eval_metric':'AUC'}
cat_dims=[]

bestparams4 = catboost_param_tune(params4, X_tr, y_tr, cat_dims, 4, skf)





OUT: best: 0.8300000330943708 {'thread_count': 4, 'iterations': 250, 'depth': 2, 'l2_leaf_reg': 7, 'scale_pos_weight': 4.624193250968099, 'learning_rate': 0.1, 'eval_metric': 'AUC'}

In [407]:
clf_cb4 = cb.CatBoostClassifier(**bestparams4)
clf_cb4.fit(X_tr, np.ravel(y_tr), cat_features = cat_dims)



In [417]:
res_cb4 = clf_cb4.predict_proba(X_te)[:,1]


yt = clf_cb4.predict_proba(X_tr)[:,1]
tr = Find_Optimal_Cutoff(y_tr, yt)[0]

res_cb4 = np.where(res_cb4 < tr , 0, res_cb4)
res_cb4 = np.where(res_cb4 >= tr , 1, res_cb4)
print(np.sum(res_cb4)/len(res_cb4))

0.37229956793086894


# correlation among all ensemble predictions 

In [414]:
# base = voting 
base=t_1
print('base - ens1_catb',matthews_corrcoef(base, res_cb3))
print('base - ens1_extr',matthews_corrcoef(base, res_et))
print('ens1_extr - ens1_catb',matthews_corrcoef(res_et, res_cb3))

#print('base - ens2_lr',matthews_corrcoef(base, y_lr))
print('base - ens2_catb',matthews_corrcoef(base, res_cb4))
#print('ens2_lr - ens2_catb',matthews_corrcoef(y_lr, res_cb4))

print('ens1_catb - ens2_catb',matthews_corrcoef(res_cb3, res_cb4))

print('ens1_extr - ens2_catb',matthews_corrcoef(res_et, res_cb4))
print('ens1_extr - ens2_catb',matthews_corrcoef(base, res_cb3))

base - ens1_catb 0.84642030854424
base - ens1_extr 0.8965252541591218
ens1_extr - ens1_catb 0.7825329198686126
base - ens2_catb 0.8507852823046459
ens1_catb - ens2_catb 0.9117924220053101
ens1_extr - ens2_catb 0.7776133680768467
ens1_extr - ens2_catb 0.84642030854424
